In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import xgboost as xgb
import pandas as pd

from helper import util_two_class

from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

# 0. Get folder names for train and validation

In [2]:
folder_train = '/path/to/9_FINAL/data/machine_learning/two_class/distance_prepared/training_downsampled_mean/training_'

In [3]:
folder_validation = '/path/to/9_FINAL/data/machine_learning/two_class/distance_prepared/validation/cross_validation/means/val_fold_'

In [4]:
# sanity check
validation_fold_0 = pd.read_csv(folder_validation+'0.csv',
                               sep=";")
validation_fold_0

,id,frequency,pidspread,pldspread,label,p1,p10,p11,p12a,p12b,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,343190981,1,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,453050043,1,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-8.0,0.0,0.0,0.0,0.0,0.0,0.0
2,344166346,2,1,2,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,240868865,17,3,16,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
4,297345077,3,3,3,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1322851,203939908,1,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1322852,376449341,1,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0.0
1322853,28222046,2,1,2,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1322854,393022542,1,1,1,0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
# get labels
validation_fold_0.label.value_counts()

0    1300010
1      22846
Name: label, dtype: int64

In [6]:
# get baseline
negatives = [0] * len(validation_fold_0)
print(f'Precision {precision_score(validation_fold_0.label.to_list(), negatives)}, Recall {recall_score(validation_fold_0.label.to_list(), negatives)}, F1 {f1_score(validation_fold_0.label.to_list(), negatives)}')

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision 0.0, Recall 0.0, F1 0.0


In [7]:
# flip labels 
conf_matrix = confusion_matrix(validation_fold_0.label.to_list(), 
                               negatives)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
f1 = (2*precision*recall)/(precision+recall)
print(f'Precision {precision}, Recall {recall}, F1 {f1}')        

Precision 0.9827297906952835, Recall 1.0, F1 0.9912896808300539


In [8]:
# get goldstandard
goldstandard = pd.read_csv('/path/to/9_FINAL/data/goldstandard/goldstandard_distance_mean.csv',
                          sep=";")
goldstandard

,id,p1,p10,p12a,p12c,p15b,p16,p2,p20a,p20c,...,p3a,p4,p43,p5,p7,p8a,p8b,p8c,p8d,label
0,1231983,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,6969816,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,14220439,0.0,0.0,0.0,0.0,0.0,0.0,-9.0,0.0,0.0,...,-3.0,0.0,0.0,-9.0,0.0,0.0,0.0,0.0,0.0,0
3,14431636,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,19194895,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,499955920,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0
135,501384948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0
136,507206333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
137,508437025,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [9]:
# get the difference of columns between goldstandard and validation set
validation_fold_0.columns 

Index(['id', 'frequency', 'pidspread', 'pldspread', 'label', 'p1', 'p10',
       'p11', 'p12a', 'p12b', 'p12c', 'p13', 'p14', 'p15a', 'p15b', 'p16',
       'p2', 'p20a', 'p20b', 'p20c', 'p20d', 'p21a', 'p21b', 'p21c', 'p21d',
       'p22a', 'p22b', 'p23a', 'p23b', 'p23c', 'p23d', 'p23e', 'p24', 'p25',
       'p26', 'p27a', 'p27b', 'p28a', 'p28b', 'p28c', 'p28d', 'p29a', 'p29c',
       'p30a', 'p30b', 'p31a', 'p31b', 'p34', 'p36', 'p37', 'p38', 'p39',
       'p3a', 'p4', 'p42', 'p43', 'p5', 'p6', 'p7', 'p8a', 'p8b', 'p8c',
       'p8d'],
      dtype='object')

In [10]:
goldstandard.columns

Index(['id', 'p1', 'p10', 'p12a', 'p12c', 'p15b', 'p16', 'p2', 'p20a', 'p20c',
       'p21a', 'p21c', 'p21d', 'p23a', 'p23b', 'p23c', 'p23d', 'p23e', 'p24',
       'p25', 'p26', 'p27a', 'p27b', 'p28a', 'p28b', 'p28c', 'p28d', 'p34',
       'p36', 'p3a', 'p4', 'p43', 'p5', 'p7', 'p8a', 'p8b', 'p8c', 'p8d',
       'label'],
      dtype='object')

In [11]:
# get frequency, pidspread and pldspread
meta_gold = pd.read_csv('/path/to/9_FINAL/data/goldstandard/goldstandard_all.csv',
                        sep=";", usecols=['_id', 'frequency', 'pidspread', 'pldspread'])
meta_gold.columns = ['id', 'frequency', 'pidspread', 'pldspread']
meta_gold

,id,frequency,pidspread,pldspread
0,141663461,3,2,3
1,147713771,3,2,3
2,270852864,1,1,1
3,516170918,1,1,1
4,387574181,1,1,1
...,...,...,...,...
134,420639686,2,1,2
135,199660983,3,3,3
136,218064825,5,2,5
137,89573263,2,1,2


In [12]:
# merge data to goldstandard
goldstandard = pd.merge(meta_gold, goldstandard)
goldstandard

,id,frequency,pidspread,pldspread,p1,p10,p12a,p12c,p15b,p16,...,p3a,p4,p43,p5,p7,p8a,p8b,p8c,p8d,label
0,141663461,3,2,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,2.333333,0.0,0.0,0.0,0
1,147713771,3,2,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0
2,270852864,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0
3,516170918,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,2.0,0.0,1
4,387574181,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,2.000000,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,420639686,2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,8.500000,0.0,0.0,0.0,0
135,199660983,3,3,3,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-4.666667,5.0,0.000000,0.0,0.0,0.0,0
136,218064825,5,2,5,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0
137,89573263,2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,2.000000,0.0,0.0,0.0,0


In [13]:
# get pid column with 0 for the columns without values
pad_with_zeros = [0] * len(goldstandard)

In [14]:
# get difference
difference = list(set(validation_fold_0.columns.to_list()) - set(goldstandard.columns.to_list()))
difference.sort()
difference

['p11',
 'p12b',
 'p13',
 'p14',
 'p15a',
 'p20b',
 'p20d',
 'p21b',
 'p22a',
 'p22b',
 'p29a',
 'p29c',
 'p30a',
 'p30b',
 'p31a',
 'p31b',
 'p37',
 'p38',
 'p39',
 'p42',
 'p6']

In [15]:
len(difference)

21

In [16]:
# delete label column from validation
del validation_fold_0['label']

In [17]:
# insert columns
for element in difference:
    goldstandard.insert(validation_fold_0.columns.get_loc(element), element, pad_with_zeros)
goldstandard.columns

Index(['id', 'frequency', 'pidspread', 'pldspread', 'p1', 'p10', 'p11', 'p12a',
       'p12b', 'p12c', 'p13', 'p14', 'p15a', 'p15b', 'p16', 'p2', 'p20a',
       'p20b', 'p20c', 'p20d', 'p21a', 'p21b', 'p21c', 'p21d', 'p22a', 'p22b',
       'p23a', 'p23b', 'p23c', 'p23d', 'p23e', 'p24', 'p25', 'p26', 'p27a',
       'p27b', 'p28a', 'p28b', 'p28c', 'p28d', 'p29a', 'p29c', 'p30a', 'p30b',
       'p31a', 'p31b', 'p34', 'p36', 'p37', 'p38', 'p39', 'p3a', 'p4', 'p42',
       'p43', 'p5', 'p6', 'p7', 'p8a', 'p8b', 'p8c', 'p8d', 'label'],
      dtype='object')

In [18]:
validation_fold_0.columns

Index(['id', 'frequency', 'pidspread', 'pldspread', 'p1', 'p10', 'p11', 'p12a',
       'p12b', 'p12c', 'p13', 'p14', 'p15a', 'p15b', 'p16', 'p2', 'p20a',
       'p20b', 'p20c', 'p20d', 'p21a', 'p21b', 'p21c', 'p21d', 'p22a', 'p22b',
       'p23a', 'p23b', 'p23c', 'p23d', 'p23e', 'p24', 'p25', 'p26', 'p27a',
       'p27b', 'p28a', 'p28b', 'p28c', 'p28d', 'p29a', 'p29c', 'p30a', 'p30b',
       'p31a', 'p31b', 'p34', 'p36', 'p37', 'p38', 'p39', 'p3a', 'p4', 'p42',
       'p43', 'p5', 'p6', 'p7', 'p8a', 'p8b', 'p8c', 'p8d'],
      dtype='object')

# 1. Logistic Regression

## 1.1 Distance columns

In [19]:
lr = LogisticRegression(random_state=42)

In [20]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                lr)

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[727508 572502]
 [  8610  14236]]
Precision 0.024262958935674865 and Recall 0.623128775277948 and F1 0.04670726265781253
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[728748 571262]
 [  8495  14351]]
Precision 0.024505945052449313 and Recall 0.6281624792086142 and F1 0.04717162536834857
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[863721 436289]
 [ 11321  11525]]
Precision 0.025736131518889538 and Recall 0.5044646765298083 and F1 0.0489737814983215
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[794686 505324]
 [ 10467  12379]]
Precision 0.02391139321193812 and Recall 0.5418453996323207 and F1 0.04580158320522284
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[853446 446566]
 [ 10989  11860]]
Precision 0.02587113296366262 and Recall 0.5190599150947525 and F1 0.0492857513895382
----------------------------------
---------------------
Overall Precision: 0.02485751233652289
Overall Recall: 0.5633322491486886
Overall F1: 0.04758800082384873
---------------------
Negative Precision: 0.9876261636425714
Negative Recall: 0.6104734097458133
Negative F1: 0.7535641407085167


## 1.2 Scaled Distance columns

In [21]:
lr = LogisticRegression(random_state=42)

In [22]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                lr, scale=True)

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[588774 711236]
 [  5709  17137]]
Precision 0.02352778040921341 and Recall 0.7501094283463189 and F1 0.045624511627102085
----------------------------------
[[1014494  285516]
 [  14564    8282]]
Precision 0.028189436279348397 and Recall 0.36251422568502145 and F1 0.052311112795442205
----------------------------------
[[1121213  178797]
 [  17249    5597]]
Precision 0.03035348221742573 and Recall 0.24498818173859757 and F1 0.05401466898282185
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[986552 313458]
 [ 14007   8839]]
Precision 0.027425014815527293 and Recall 0.3868948612448569 and F1 0.0512193496608652
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1114214  185798]
 [  17196    5653]]
Precision 0.029527137492099807 and Recall 0.24740688870410082 and F1 0.0527578161455903
----------------------------------
---------------------
Overall Precision: 0.027804570242722926
Overall Recall: 0.3983827171437791
Overall F1: 0.05118549184236433
---------------------
Negative Precision: 0.9863789793873223
Negative Recall: 0.7423397182853113
Negative F1: 0.8372993422584806


## 1.3 Distance columns + frequencies

In [9]:
lr = LogisticRegression(random_state=42)

In [10]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                lr)

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[877321 422689]
 [ 10639  12207]]
Precision 0.028068779662264082 and Recall 0.5343167294055853 and F1 0.05333572186952476
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[924668 375342]
 [ 11418  11428]]
Precision 0.02954727615895752 and Recall 0.5002188566926377 and F1 0.05579860161712434
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[517021 782989]
 [  5638  17208]]
Precision 0.021504704466525117 and Recall 0.7532171933817736 and F1 0.04181555520185458
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[908375 391635]
 [ 11164  11682]]
Precision 0.02896480931872448 and Recall 0.5113367766786309 and F1 0.054824093128685504
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[498844 801168]
 [  4696  18153]]
Precision 0.022156151252073365 and Recall 0.7944767823537136 and F1 0.04311006091406723
----------------------------------
---------------------
Overall Precision: 0.026048344171708916
Overall Recall: 0.6187132677024682
Overall F1: 0.04977680654625129
---------------------
Negative Precision: 0.9887134041661483
Negative Recall: 0.5732614722278715
Negative F1: 0.7136024419828836


## 1.4 Scaled distance columns + frequencies

In [11]:
lr = LogisticRegression(random_state=42)

In [12]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                lr, scale=True)

/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[311088 988922]
 [  1619  21227]]
Precision 0.021013731637609896 and Recall 0.9291342029239255 and F1 0.04109797240064086
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[ 259826 1040184]
 [   1230   21616]]
Precision 0.0203578828404596 and Recall 0.9461612536111355 and F1 0.03985816570567725
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[ 270796 1029214]
 [   1679   21167]]
Precision 0.020151735417910262 and Recall 0.9265079226122734 and F1 0.03944552270861616
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[  63933 1236077]
 [    202   22644]]
Precision 0.01798968953405878 and Recall 0.9911581896174385 and F1 0.03533798857180311
----------------------------------


/home/linda/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[[1264339   35673]
 [  20733    2116]]
Precision 0.05599513085818624 and Recall 0.09260799159700643 and F1 0.06979122002704576
----------------------------------
---------------------
Overall Precision: 0.027101634057644958
Overall Recall: 0.7771139120723559
Overall F1: 0.04510617388275663
---------------------
Negative Precision: 0.992933126172906
Negative Recall: 0.3338405173623329
Negative F1: 0.42700160111668317


# 2. Naive Bayes

## 2.1 Distance encoded columns

In [65]:
nb = MultinomialNB()

In [ ]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                nb)

## 2.2 Scaled Distance encoded columns

In [67]:
nb = MultinomialNB()

In [68]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                nb, scale=True)

[[1091322  208688]
 [  16850    5996]]
Precision 0.027929421847925323 and Recall 0.2624529458110829 and F1 0.05048625436786931
----------------------------------
[[1077502  222508]
 [  16428    6418]]
Precision 0.028035260302455817 and Recall 0.28092445066970145 and F1 0.050982635082535
----------------------------------
[[1144654  155356]
 [  18149    4697]]
Precision 0.029346528962281244 and Recall 0.2055939770638186 and F1 0.051361680490325266
----------------------------------
[[1092094  207916]
 [  17158    5688]]
Precision 0.026628714818074568 and Recall 0.248971373544603 and F1 0.04811165151194756
----------------------------------
[[1169611  130401]
 [  18923    3926]]
Precision 0.029227184408197906 and Recall 0.17182371219747034 and F1 0.04995673639741436
----------------------------------
---------------------
Overall Precision: 0.028233422067786968
Overall Recall: 0.23395329185733527
Overall F1: 0.050179791570018305
---------------------
Negative Precision: 0.984556002834968

In [72]:
nb = MultinomialNB()
train = folder_train = pd.read_csv('/path/to/9_FINAL/data/machine_learning/two_class/distance_prepared/training_downsampled_mean/training_0.csv',
                                   sep=";", index_col=0)
train_new = train.drop(columns=['frequency', 'pidspread', 'pldspread', 'id'])
y_train = train_new['label'].to_list()
X_train = train_new.drop(columns=['label'])
X_train

,p1,p10,p11,p12a,p12b,p12c,p13,p14,p15a,p15b,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,13.0,-7.0,0.0,0.0,9.0,9.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182769,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0,0.0
182770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0
182771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
182772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
# scale instances
scaler = MinMaxScaler()
# scale train and validation
X_train = scaler.fit_transform(X_train)
X_train

array([[0.        , 0.96774194, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.96774194, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.96774194, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.96774194, 0.        , ..., 0.        , 0.16      ,
        0.        ],
       [0.        , 0.96774194, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.96774194, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [75]:
goldstandard

,id,frequency,pidspread,pldspread,p1,p10,p11,p12a,p12b,p12c,...,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d,label
0,141663461,3,2,3,0.0,0.0,0,0.0,0,0.0,...,0,0.0,0.000000,0,0.0,2.333333,0.0,0.0,0.0,0
1,147713771,3,2,3,0.0,0.0,0,0.0,0,0.0,...,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0,0
2,270852864,1,1,1,0.0,0.0,0,0.0,0,0.0,...,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0,0
3,516170918,1,1,1,0.0,0.0,0,0.0,0,0.0,...,0,0.0,0.000000,0,0.0,0.000000,0.0,2.0,0.0,1
4,387574181,1,1,1,0.0,0.0,0,0.0,0,0.0,...,0,0.0,0.000000,0,0.0,2.000000,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,420639686,2,1,2,0.0,0.0,0,0.0,0,0.0,...,0,0.0,0.000000,0,0.0,8.500000,0.0,0.0,0.0,0
135,199660983,3,3,3,5.0,0.0,0,0.0,0,0.0,...,0,0.0,-4.666667,0,5.0,0.000000,0.0,0.0,0.0,0
136,218064825,5,2,5,0.0,0.0,0,0.0,0,0.0,...,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0,0
137,89573263,2,1,2,0.0,0.0,0,0.0,0,0.0,...,0,0.0,0.000000,0,0.0,2.000000,0.0,0.0,0.0,0


In [74]:
goldstandard_new = goldstandard.drop(columns=['frequency', 'pidspread', 'pldspread', 'id'])
y_gold = goldstandard_new['label'].to_list()
X_gold = goldstandard_new.drop(columns=['label'])
X_gold = scaler.fit_transform(X_gold)
X_gold

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

In [82]:
# fit model
nb.fit(X_train, y_train)
predictions = nb.predict(X_gold)

In [83]:
# get metrics
print(f'Precision {precision_score(y_gold, predictions)}, Recall {recall_score(y_gold, predictions)}, F1 {f1_score(y_gold, predictions)}')

Precision 0.047619047619047616, Recall 0.4, F1 0.0851063829787234


In [84]:
# flip labels
conf_matrix = confusion_matrix(y_gold, predictions)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
f1 = (2*precision*recall)/(precision+recall)
print(f'Precision {precision}, Recall {recall}, F1 {f1}')

Precision 0.9690721649484536, Recall 0.7014925373134329, F1 0.8138528138528138


## 2.4 Scaled Distance columns + frequencies

In [19]:
nb = MultinomialNB()

In [22]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                nb,scale=True)

[[1078741  221269]
 [  16785    6061]]
Precision 0.026661681256323407 and Recall 0.2652980828153725 and F1 0.04845388846252239
----------------------------------
[[1064140  235870]
 [  16531    6315]]
Precision 0.02607510787208126 and Recall 0.2764160028013657 and F1 0.04765480264572823
----------------------------------
[[1107990  192020]
 [  17548    5298]]
Precision 0.026850059295147933 and Recall 0.23190055151886543 and F1 0.048127759306698645
----------------------------------
[[1076114  223896]
 [  16994    5852]]
Precision 0.02547138604035726 and Recall 0.25614987306311826 and F1 0.04633522569815594
----------------------------------
[[1119979  180033]
 [  17815    5034]]
Precision 0.027200959652450195 and Recall 0.22031598757057202 and F1 0.048423401758402436
----------------------------------
---------------------
Overall Precision: 0.02645183882327201
Overall Recall: 0.2500160995538588
Overall F1: 0.04779901557430152
---------------------
Negative Precision: 0.984517372094321

# 3. XGBoost

## 3.1 Distance columns

In [23]:
xg = xgb.XGBRFClassifier(random_state=42)

In [24]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                xg)

[[870751 429259]
 [  7860  14986]]
Precision 0.03373363797003905 and Recall 0.6559572791735971 and F1 0.06416736781483694
----------------------------------
[[905370 394640]
 [  8829  14017]]
Precision 0.03430015881289199 and Recall 0.6135428521404185 and F1 0.06496826209783015
----------------------------------
[[1001261  298749]
 [  11620   11226]]
Precision 0.03621582385676264 and Recall 0.49137704631007617 and F1 0.06745968553666985
----------------------------------
[[874486 425524]
 [  7860  14986]]
Precision 0.03401965903157703 and Recall 0.6559572791735971 and F1 0.06468460535743575
----------------------------------
[[972825 327187]
 [ 10830  12019]]
Precision 0.03543274588303273 and Recall 0.5260186441419756 and F1 0.06639322754830068
----------------------------------
---------------------
Overall Precision: 0.034740405110860684
Overall Recall: 0.5885706201879328
Overall F1: 0.06553462967101467
---------------------
Negative Precision: 0.9900012345322315
Negative Recall: 0.7

## 3.2 Scaled Distance columns

In [25]:
xg = xgb.XGBRFClassifier(random_state=42)

In [27]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                xg, scale=True)

[[802025 497985]
 [  6920  15926]]
Precision 0.030989801736098273 and Recall 0.6971023373894774 and F1 0.05934156424601822
----------------------------------
[[770590 529420]
 [  7220  15626]]
Precision 0.028669139852416127 and Recall 0.6839709358312177 and F1 0.05503159051368922
----------------------------------
[[955925 344085]
 [ 10865  11981]]
Precision 0.03364825622216106 and Recall 0.524424406898363 and F1 0.06323895785828901
----------------------------------
[[794583 505427]
 [  6777  16069]]
Precision 0.03081327565312102 and Recall 0.7033616387989144 and F1 0.05904008876772324
----------------------------------
[[933063 366949]
 [ 10483  12366]]
Precision 0.03260087262565414 and Recall 0.5412053043896888 and F1 0.061497299609114695
----------------------------------
---------------------
Overall Precision: 0.03134426921789012
Overall Recall: 0.6300129246615322
Overall F1: 0.05962990019896688
---------------------
Negative Precision: 0.9902715729946678
Negative Recall: 0.65479

## 3.3 Distance columns + frequencies

In [146]:
xg = xgb.XGBRFClassifier(random_state=42)

In [ ]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                xg)

In [147]:
# get features
train_new = train.drop(columns=['id'])
X_train = train_new.drop(columns=['label'])
X_train

,frequency,pidspread,pldspread,p1,p10,p11,p12a,p12b,p12c,p13,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0
4,25,12,19,4.0,0.0,0.0,11.0,0.0,0.0,0.0,...,0.0,0.0,13.0,-7.0,0.0,0.0,9.0,9.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182769,7,4,7,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0,0.0
182770,5,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0
182771,4,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
182772,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [148]:
goldstandard_new = goldstandard.drop(columns=['id'])
y_gold = goldstandard_new['label'].to_list()
X_gold = goldstandard_new.drop(columns=['label'])
X_gold

,frequency,pidspread,pldspread,p1,p10,p11,p12a,p12b,p12c,p13,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,3,2,3,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,2.333333,0.0,0.0,0.0
1,3,2,3,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
2,1,1,1,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
3,1,1,1,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,2.0,0.0
4,1,1,1,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,2.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,2,1,2,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,8.500000,0.0,0.0,0.0
135,3,3,3,5.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,-4.666667,0,5.0,0.000000,0.0,0.0,0.0
136,5,2,5,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
137,2,1,2,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,2.000000,0.0,0.0,0.0


In [149]:
# fit model
xg.fit(X_train, y_train)
predictions = xg.predict(X_gold)

In [150]:
# get metrics
print(f'Precision {precision_score(y_gold, predictions)}, Recall {recall_score(y_gold, predictions)}, F1 {f1_score(y_gold, predictions)}')

Precision 0.07407407407407407, Recall 0.4, F1 0.125


In [151]:
# flip labels
conf_matrix = confusion_matrix(y_gold, predictions)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
f1 = (2*precision*recall)/(precision+recall)
print(f'Precision {precision}, Recall {recall}, F1 {f1}')

Precision 0.9732142857142857, Recall 0.8134328358208955, F1 0.8861788617886178


## 3.4 Scaled Distance columns + frequencies

In [30]:
xg = xgb.XGBRFClassifier(random_state=42)

In [31]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                xg, scale=True)

[[802806 497204]
 [  6924  15922]]
Precision 0.03102941577702163 and Recall 0.6969272520353672 and F1 0.05941355145418044
----------------------------------
[[766734 533276]
 [  7152  15694]]
Precision 0.028588083137512068 and Recall 0.6869473868510899 and F1 0.054891783370874546
----------------------------------
[[957616 342394]
 [ 10830  12016]]
Precision 0.03390423520781016 and Recall 0.5259564037468266 and F1 0.06370210149076488
----------------------------------
[[797621 502389]
 [  6807  16039]]
Precision 0.03093775799146651 and Recall 0.7020484986430885 and F1 0.05926388483466784
----------------------------------
[[932723 367289]
 [ 10478  12371]]
Precision 0.03258441763683296 and Recall 0.5414241323471487 and F1 0.06146943298162278
----------------------------------
---------------------
Overall Precision: 0.03140878195012866
Overall Recall: 0.6306607347247042
Overall F1: 0.0597481508264221
---------------------
Negative Precision: 0.9902907147939175
Negative Recall: 0.654994

# 4. Random Forest

## 4.1 Distance encoded columns

In [140]:
rf = RandomForestClassifier(random_state=42)

In [33]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                rf)

[[877077 422933]
 [  7512  15334]]
Precision 0.034987804238055795 and Recall 0.6711897049811784 and F1 0.06650864321760609
----------------------------------
[[875434 424576]
 [  7581  15265]]
Precision 0.03470572320452164 and Recall 0.6681694826227786 and F1 0.06598413182129605
----------------------------------
[[971429 328581]
 [ 10608  12238]]
Precision 0.035907622521044895 and Recall 0.5356736408999387 and F1 0.06730369983363811
----------------------------------
[[874043 425967]
 [  7329  15517]]
Precision 0.03514736660898243 and Recall 0.6791998599317167 and F1 0.06683608640406608
----------------------------------
[[971749 328263]
 [ 10397  12452]]
Precision 0.03654667390634401 and Recall 0.5449691452579981 and F1 0.0684996314266539
----------------------------------
---------------------
Overall Precision: 0.03545903809578975
Overall Recall: 0.6198403667387221
Overall F1: 0.06702643854065204
---------------------
Negative Precision: 0.9906438156944276
Negative Recall: 0.703030

In [141]:
# get features
train_new = train.drop(columns=['frequency', 'pidspread', 'pldspread','id'])
X_train = train_new.drop(columns=['label'])
X_train

,p1,p10,p11,p12a,p12b,p12c,p13,p14,p15a,p15b,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,13.0,-7.0,0.0,0.0,9.0,9.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182769,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0,0.0
182770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0
182771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
182772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
goldstandard_new = goldstandard.drop(columns=['frequency', 'pidspread', 'pldspread','id'])
y_gold = goldstandard_new['label'].to_list()
X_gold = goldstandard_new.drop(columns=['label'])
X_gold

,p1,p10,p11,p12a,p12b,p12c,p13,p14,p15a,p15b,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,2.333333,0.0,0.0,0.0
1,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,2.0,0.0
4,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,2.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,8.500000,0.0,0.0,0.0
135,5.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,-4.666667,0,5.0,0.000000,0.0,0.0,0.0
136,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
137,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,2.000000,0.0,0.0,0.0


In [143]:
# fit model
rf.fit(X_train, y_train)
predictions = rf.predict(X_gold)

In [144]:
# get metrics
print(f'Precision {precision_score(y_gold, predictions)}, Recall {recall_score(y_gold, predictions)}, F1 {f1_score(y_gold, predictions)}')

Precision 0.10344827586206896, Recall 0.6, F1 0.17647058823529413


In [145]:
# flip labels
conf_matrix = confusion_matrix(y_gold, predictions)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
f1 = (2*precision*recall)/(precision+recall)
print(f'Precision {precision}, Recall {recall}, F1 {f1}')

Precision 0.9818181818181818, Recall 0.8059701492537313, F1 0.8852459016393442


## 4.2 Scaled Distance encoded columns

In [34]:
rf = RandomForestClassifier(random_state=42)

In [35]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                rf, scale=True)

[[841132 458878]
 [  8857  13989]]
Precision 0.029583371222775115 and Recall 0.6123172546616475 and F1 0.056439915838398434
----------------------------------
[[719287 580723]
 [  7563  15283]]
Precision 0.02564235930510767 and Recall 0.6689573667162741 and F1 0.04939145385326378
----------------------------------
[[544983 755027]
 [  4533  18313]]
Precision 0.023680399306902526 and Recall 0.8015845224546967 and F1 0.04600181364655997
----------------------------------
[[630831 669179]
 [  4565  18281]]
Precision 0.026592092630843975 and Recall 0.8001838396218156 and F1 0.05147359025546736
----------------------------------
[[919200 380812]
 [ 10307  12542]]
Precision 0.031884765376734445 and Recall 0.5489080484922754 and F1 0.06026866697260712
----------------------------------
---------------------
Overall Precision: 0.027476597568472743
Overall Recall: 0.686390206389342
Overall F1: 0.05271508811325933
---------------------
Negative Precision: 0.9905304889836394
Negative Recall: 0.56

## 4.3 Distance columns + frequencies

In [36]:
rf = RandomForestClassifier(random_state=42)

In [37]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                rf)

[[873056 426954]
 [  7428  15418]]
Precision 0.0348530196305372 and Recall 0.674866497417491 and F1 0.06628290392891076
----------------------------------
[[877908 422102]
 [  7586  15260]]
Precision 0.03489100562005844 and Recall 0.667950625930141 and F1 0.06631783889024093
----------------------------------
[[926660 373350]
 [  9871  12975]]
Precision 0.033585711512327705 and Recall 0.56793311739473 and F1 0.06342091692715271
----------------------------------
[[875487 424523]
 [  7394  15452]]
Precision 0.03512017728280016 and Recall 0.6763547229274272 and F1 0.06677311530807807
----------------------------------
[[948297 351715]
 [ 10015  12834]]
Precision 0.03520514389012177 and Recall 0.5616876012079304 and F1 0.06625744066825333
----------------------------------
---------------------
Overall Precision: 0.034731011587169056
Overall Recall: 0.629758512975544
Overall F1: 0.06581044314452716
---------------------
Negative Precision: 0.9907262557340444
Negative Recall: 0.69251875617

## 4.4 scaled Distance columns + frequencies

In [38]:
rf = RandomForestClassifier(random_state=42)

In [39]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                rf, scale=True)

[[872617 427393]
 [  8503  14343]]
Precision 0.03246961986344785 and Recall 0.6278123085003939 and F1 0.0617458274319711
----------------------------------
[[892719 407291]
 [  9945  12901]]
Precision 0.030702631178128093 and Recall 0.5646940383436926 and F1 0.058238796672068764
----------------------------------
[[882278 417732]
 [  9223  13623]]
Precision 0.03158187571721668 and Recall 0.5962969447605708 and F1 0.059986657889348545
----------------------------------
[[779680 520330]
 [  6696  16150]]
Precision 0.030103638532657323 and Recall 0.7069071172196446 and F1 0.05774807536213228
----------------------------------
[[871777 428235]
 [  9149  13700]]
Precision 0.031000033941643003 and Recall 0.5995886034399754 and F1 0.0589521153912355
----------------------------------
---------------------
Overall Precision: 0.03117155984661859
Overall Recall: 0.6190598024528555
Overall F1: 0.05933429454935124
---------------------
Negative Precision: 0.9900172494083683
Negative Recall: 0.6613

# 5. MLPClassifier

## 5.1 Distance encoded columns

In [40]:
nnet = MLPClassifier(random_state=42)

In [41]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                nnet)

[[838190 461820]
 [  6933  15913]]
Precision 0.033309400857801326 and Recall 0.6965333099886194 and F1 0.06357837624031372
----------------------------------
[[842648 457362]
 [  7087  15759]]
Precision 0.03330860393007286 and Recall 0.6897925238553795 and F1 0.06354858286942479
----------------------------------
[[952509 347501]
 [ 10242  12604]]
Precision 0.035000902514544366 and Recall 0.5516939508010155 and F1 0.06582565393483761
----------------------------------
[[838599 461411]
 [  6754  16092]]
Precision 0.03370031183050159 and Recall 0.7043683795850477 and F1 0.06432310247447282
----------------------------------
[[947695 352317]
 [  9993  12856]]
Precision 0.03520523149301838 and Recall 0.5626504442207536 and F1 0.06626428398389783
----------------------------------
---------------------
Overall Precision: 0.034104890125187706
Overall Recall: 0.6410077216901631
Overall F1: 0.06470799990058936
---------------------
Negative Precision: 0.9908787763795989
Negative Recall: 0.6799

## 5.2 Scaled Distance encoded columns

In [42]:
nnet = MLPClassifier(random_state=42)

In [43]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                nnet, scale=True)

[[740921 559089]
 [  6135  16711]]
Precision 0.02902222994095172 and Recall 0.7314628381335901 and F1 0.055829321502189944
----------------------------------
[[734858 565152]
 [  6317  16529]]
Precision 0.028415918690828823 and Recall 0.7234964545215793 and F1 0.05468407531838942
----------------------------------
[[924771 375239]
 [ 10055  12791]]
Precision 0.032963946086642784 and Recall 0.559879191105664 and F1 0.0622620936730303
----------------------------------
[[413408 886602]
 [  2081  20765]]
Precision 0.02288489662947848 and Recall 0.9089118445242056 and F1 0.044645688675604416
----------------------------------
[[1137992  162020]
 [  15409    7440]]
Precision 0.043904166174908534 and Recall 0.3256160007002495 and F1 0.07737547384677784
----------------------------------
---------------------
Overall Precision: 0.03143823150456207
Overall Recall: 0.6498732657970576
Overall F1: 0.05895933060319839
---------------------
Negative Precision: 0.9908281242956536
Negative Recall: 0.

## 5.3 Distance encoded columns + frequencies

In [132]:
nnet = MLPClassifier(random_state=42)

In [45]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                nnet)

[[879836 420174]
 [  7530  15316]]
Precision 0.03516957909481274 and Recall 0.6704018208876827 and F1 0.06683306569852684
----------------------------------
[[858671 441339]
 [  7169  15677]]
Precision 0.0343029565704483 and Recall 0.6862032740961218 and F1 0.0653396184736445
----------------------------------
[[960592 339418]
 [ 10384  12462]]
Precision 0.03541548255086961 and Recall 0.5454784207301059 and F1 0.06651259853866559
----------------------------------
[[848395 451615]
 [  6748  16098]]
Precision 0.034418542995383924 and Recall 0.7046310076162129 and F1 0.06563124924830653
----------------------------------
[[924695 375317]
 [  9553  13296]]
Precision 0.0342139866653972 and Recall 0.58190730447722 and F1 0.06462808230164632
----------------------------------
---------------------
Overall Precision: 0.03470410957538236
Overall Recall: 0.6377243655614687
Overall F1: 0.06578892285215795
---------------------
Negative Precision: 0.9908847165394329
Negative Recall: 0.68802356557

In [134]:
# get features
train_new = train.drop(columns=['id'])
X_train = train_new.drop(columns=['label'])
X_train

,frequency,pidspread,pldspread,p1,p10,p11,p12a,p12b,p12c,p13,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2,1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0
4,25,12,19,4.0,0.0,0.0,11.0,0.0,0.0,0.0,...,0.0,0.0,13.0,-7.0,0.0,0.0,9.0,9.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182769,7,4,7,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0,0.0
182770,5,2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0
182771,4,1,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
182772,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [135]:
goldstandard_new = goldstandard.drop(columns=['id'])
y_gold = goldstandard_new['label'].to_list()
X_gold = goldstandard_new.drop(columns=['label'])
X_gold

,frequency,pidspread,pldspread,p1,p10,p11,p12a,p12b,p12c,p13,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,3,2,3,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,2.333333,0.0,0.0,0.0
1,3,2,3,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
2,1,1,1,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
3,1,1,1,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,2.0,0.0
4,1,1,1,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,2.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,2,1,2,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,8.500000,0.0,0.0,0.0
135,3,3,3,5.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,-4.666667,0,5.0,0.000000,0.0,0.0,0.0
136,5,2,5,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
137,2,1,2,0.0,0.0,0,0.0,0,0.0,0,...,0.0,0,0.0,0.000000,0,0.0,2.000000,0.0,0.0,0.0


In [136]:
# fit model
nnet.fit(X_train, y_train)
predictions = nnet.predict(X_gold)

In [137]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 0])

In [138]:
# get metrics
print(f'Precision {precision_score(y_gold, list(predictions))}, Recall {recall_score(y_gold, list(predictions))}, F1 {f1_score(y_gold, list(predictions))}')

Precision 0.09090909090909091, Recall 0.6, F1 0.15789473684210525


In [139]:
# flip labels
conf_matrix = confusion_matrix(y_gold, predictions)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
f1 = (2*precision*recall)/(precision+recall)
print(f'Precision {precision}, Recall {recall}, F1 {f1}')

Precision 0.9811320754716981, Recall 0.7761194029850746, F1 0.8666666666666666


## 5.4 Scaled Distance encoded columns + frequencies

In [46]:
nnet = MLPClassifier(random_state=42)

In [47]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                nnet, scale=True)

[[665461 634549]
 [  5231  17615]]
Precision 0.027010077219840407 and Recall 0.7710321281624792 and F1 0.05219181938045362
----------------------------------
[[501263 798747]
 [  3146  19700]]
Precision 0.024069976430972317 and Recall 0.8622953689923838 and F1 0.04683267304018932
----------------------------------
[[797592 502418]
 [  8240  14606]]
Precision 0.028250139258525715 and Recall 0.639324170533135 and F1 0.054109322614703546
----------------------------------
[[305490 994520]
 [  1190  21656]]
Precision 0.021311268914046385 and Recall 0.9479121071522367 and F1 0.04168535411184749
----------------------------------
[[1226926   73086]
 [  18244    4605]]
Precision 0.05927327489670618 and Recall 0.2015405488205173 and F1 0.09160533121145813
----------------------------------
---------------------
Overall Precision: 0.0319829473440182
Overall Recall: 0.6844208647321505
Overall F1: 0.05728490007173041
---------------------
Negative Precision: 0.9914412107190953
Negative Recall: 0.

# 6. DecisionTreeClassifier

## 6.1 Distance encoded columns

In [125]:
dt = DecisionTreeClassifier(random_state=42)

In [49]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                dt)

[[858249 441761]
 [  8384  14462]]
Precision 0.03169941015687504 and Recall 0.6330210977851702 and F1 0.060375436523757535
----------------------------------
[[856433 443577]
 [  8380  14466]]
Precision 0.03158218769853485 and Recall 0.6331961831392804 and F1 0.06016357205093067
----------------------------------
[[960544 339466]
 [ 11284  11562]]
Precision 0.03293754344382784 and Recall 0.5060842160553269 and F1 0.061849714074795246
----------------------------------
[[855342 444668]
 [  8032  14814]]
Precision 0.03224065360558194 and Recall 0.6484286089468616 and F1 0.061427078668457986
----------------------------------
[[960965 339047]
 [ 11053  11796]]
Precision 0.03362187645186023 and Recall 0.5162589172392665 and F1 0.06313220513149866
----------------------------------
---------------------
Overall Precision: 0.032416334271335986
Overall Recall: 0.587397804633181
Overall F1: 0.06138960128988802
---------------------
Negative Precision: 0.9896700977032967
Negative Recall: 0.6909

In [126]:
# get features
train_new = train.drop(columns=['frequency', 'pidspread', 
                         'pldspread','id'])
X_train = train_new.drop(columns=['label'])
X_train

,p1,p10,p11,p12a,p12b,p12c,p13,p14,p15a,p15b,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-7.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,13.0,-7.0,0.0,0.0,9.0,9.0,9.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182769,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,2.0,0.0
182770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0
182771,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0
182772,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [127]:
goldstandard_new = goldstandard.drop(columns=['frequency', 'pidspread', 
                 'pldspread','id'])
y_gold = goldstandard_new['label'].to_list()
X_gold = goldstandard_new.drop(columns=['label'])
X_gold

,p1,p10,p11,p12a,p12b,p12c,p13,p14,p15a,p15b,...,p4,p42,p43,p5,p6,p7,p8a,p8b,p8c,p8d
0,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,2.333333,0.0,0.0,0.0
1,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
3,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,2.0,0.0
4,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,2.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,8.500000,0.0,0.0,0.0
135,5.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,-4.666667,0,5.0,0.000000,0.0,0.0,0.0
136,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,0.000000,0.0,0.0,0.0
137,0.0,0.0,0,0.0,0,0.0,0,0,0,0.0,...,0.0,0,0.0,0.000000,0,0.0,2.000000,0.0,0.0,0.0


In [128]:
# fit model
dt.fit(X_train, y_train)
predictions = dt.predict(X_gold)

In [129]:
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0])

In [130]:
# get metrics
print(f'Precision {precision_score(y_gold, predictions)}, Recall {recall_score(y_gold, predictions)}, F1 {f1_score(y_gold, predictions)}')

Precision 0.09090909090909091, Recall 0.6, F1 0.15789473684210525


In [131]:
# flip labels
conf_matrix = confusion_matrix(y_gold, predictions)
precision = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[1,0])
recall = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
f1 = (2*precision*recall)/(precision+recall)
print(f'Precision {precision}, Recall {recall}, F1 {f1}')

Precision 0.9811320754716981, Recall 0.7761194029850746, F1 0.8666666666666666


## 6.2 Scaled Distance encoded columns

In [50]:
dt = DecisionTreeClassifier(random_state=42)

In [51]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['frequency', 'pidspread', 
                 'pldspread', 'id'],
                dt, scale=True)

[[671488 628522]
 [  7339  15507]]
Precision 0.024078108283943734 and Recall 0.6787621465464414 and F1 0.046506466729147135
----------------------------------
[[684068 615942]
 [  8613  14233]]
Precision 0.022585789661601935 and Recall 0.6229974612623654 and F1 0.04359124744839752
----------------------------------
[[518491 781519]
 [  5341  17505]]
Precision 0.021907977732834056 and Recall 0.7662172809244506 and F1 0.042597977782374334
----------------------------------
[[641723 658287]
 [  6351  16495]]
Precision 0.024444931844655016 and Recall 0.7220082290116432 and F1 0.04728881294902155
----------------------------------
[[997008 303004]
 [ 13333   9516]]
Precision 0.030449251247920132 and Recall 0.4164733686375771 and F1 0.05674943122351797
----------------------------------
---------------------
Overall Precision: 0.024693211754190973
Overall Recall: 0.6412916972764956
Overall F1: 0.0473467872264917
---------------------
Negative Precision: 0.9887124099623492
Negative Recall: 0.

## 6.3 Only distance encoded columns + frequencies

In [52]:
dt = DecisionTreeClassifier(random_state=42)

In [53]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                dt)

[[852941 447069]
 [  8700  14146]]
Precision 0.030671162039395943 and Recall 0.6191893548104701 and F1 0.0584471791778309
----------------------------------
[[851486 448524]
 [  8667  14179]]
Precision 0.030643847133042145 and Recall 0.6206338089818787 and F1 0.05840399218204548
----------------------------------
[[917050 382960]
 [ 10819  12027]]
Precision 0.030449103388212776 and Recall 0.5264378884706294 and F1 0.05756845438249252
----------------------------------
[[849509 450501]
 [  8392  14454]]
Precision 0.03108687937542343 and Recall 0.63267092707695 and F1 0.059261871131875504
----------------------------------
[[932269 367743]
 [ 10788  12061]]
Precision 0.03175585301892555 and Recall 0.5278567989846383 and F1 0.05990766242894005
----------------------------------
---------------------
Overall Precision: 0.03092136899099997
Overall Recall: 0.5853577556649133
Overall F1: 0.05871783186063689
---------------------
Negative Precision: 0.9893890828143977
Negative Recall: 0.677418

## 6.4 Scaled distance encoded columns + frequencies

In [54]:
dt = DecisionTreeClassifier(random_state=42)

In [55]:
util_two_class.cross_validation(folder_validation, folder_train,
                ['id'],
                dt, scale=True)

[[817040 482970]
 [  8650  14196]]
Precision 0.02855384318316216 and Recall 0.6213779217368467 and F1 0.05459874002907625
----------------------------------
[[797806 502204]
 [  9405  13441]]
Precision 0.026066382879694365 and Recall 0.58833056114856 and F1 0.04992098289479305
----------------------------------
[[505343 794667]
 [  5561  17285]]
Precision 0.021288204228821408 and Recall 0.7565875864483936 and F1 0.041411215647378166
----------------------------------
[[795189 504821]
 [  8601  14245]]
Precision 0.027443523559624403 and Recall 0.6235227173246958 and F1 0.0525731115014984
----------------------------------
[[869373 430639]
 [ 10788  12061]]
Precision 0.0272441834199232 and Recall 0.5278567989846383 and F1 0.05181409475694288
----------------------------------
---------------------
Overall Precision: 0.02611922745424511
Overall Recall: 0.6235351171286269
Overall F1: 0.050063628965937745
---------------------
Negative Precision: 0.9888061303971767
Negative Recall: 0.582264